In [1]:
# !pip install pandas
# !pip install ipywidgets
# !jupyter nbextension enable --py widgetsnbextension
# ! python38 -m pip install --user --force-reinstall tensorflow

In [8]:
import pandas as pd
import numpy
import os
from os import walk
import os.path
from os import path

from IPython.display import display, Markdown, clear_output
# widget packages
import ipywidgets as widgets

In [4]:
def call_morpher():
    result = !python38 ../morpher/test/test_morpher.py
    print(result)

In [31]:
def get_target_img():
    target_file = "../morpher/test/morpher_output/" + destination_option.value + "_" + destination.value.replace(".jpg", "") + "_" + str(slider.value) + ".jpg"
    if path.exists(target_file):
        return target_file
    return ""

def read_img(target_file):
    output_filenames = next(walk("../morpher/test/morpher_output"), (None, None, []))[2]
    file = open(target_file, "rb")
    image = file.read()
    return image

In [37]:
# some handy functions to use along widgets

slider = widgets.FloatSlider(
         description='Alpha:',
         value=0.5,
         min=0,
         max=1.0,
         step=0.1,)

def slider_on_value_change(change):
    # Update image
    target_img_path = get_target_img()
    if target_img_path!="":
        image_show.value = read_img(target_img_path)

slider.observe(slider_on_value_change, names='value')

dts_filenames = next(walk("../morpher/test/dst"), (None, None, []))[2]
# dts_filenames
dts_filenames.append("test.jpg")

# destination image
destination = widgets.Dropdown(
       options=dts_filenames,
       value=dts_filenames[0],
       description='Destination:')

def destination_img_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # Update 'Alpha'
        img_name = destination.value
        param_file = current_path+"/../morpher/test/parameters.csv"
        df = pd.read_csv(param_file)
        # Update
        df['my_dst_img'] = [img_name]
        df = df[['percent', 'my_dst_img', 'my_src_img']]
        df.to_csv(param_file, index=False)

        # Update image
        target_img_path = get_target_img()
        if target_img_path!="":
            image_show.value = read_img(target_img_path)

destination.observe(destination_img_on_change)

# morphing_options
morphing_options = ["alphabg", "overlay", "poisson"]
destination_option = widgets.Dropdown(
       options=morphing_options,
       value=morphing_options[0],
       description='Option:')

def destination_option_on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        # Update image
        target_img_path = get_target_img()
        if target_img_path!="":
            image_show.value = read_img(target_img_path)

destination_option.observe(destination_option_on_change)

# morphing_cmd textarea
morphing_cmd = "Ready."

textarea = widgets.Textarea(
    value=morphing_cmd,
    placeholder='',
    description='Status:',
    disabled=True
)

def set_textarea(my_textarea, a_cmd):
    # morphing_cmd textarea
    my_textarea.value = a_cmd

# button
button = widgets.Button(
    description='Morph',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Morph',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
# Event
def on_button_clicked(b):

    # Update 'Alpha'
    percentage = round(float(slider.value), 1)
    param_file = current_path+"/../morpher/test/parameters.csv"
    df = pd.read_csv(param_file)
    # Update
    df['percent'] = [percentage]
    df = df[['percent', 'my_dst_img', 'my_src_img']]
    df.to_csv(param_file, index=False)
    
    # Call 
    set_textarea(textarea, "Running...")
#     call_morpher()
    set_textarea(textarea, "Finshed.")

button.on_click(on_button_clicked)

# Image

inital_img = None
target_img_path = get_target_img()
if target_img_path!="":
    inital_img = read_img(target_img_path)
    
image_show = widgets.Image(
    value=inital_img,
    format='png',
    width=500,
    height=600,
)

# box1 = widgets.VBox([slider, destination, destination_option, button])
# box1

In [38]:
box1 = widgets.VBox([slider, destination, destination_option, textarea, button, image_show])
box1